In [ ]:
from etils import epath
import numpy as np
from matplotlib import pyplot as plt
import torch
import torchio as tio
from torchsummary import summary

from codebase.preprocessor.images import multi_modal_processor
import codebase.terminology as term
import codebase.codebase_settings as cbs

%load_ext autoreload
%autoreload 2

<h3> Preprocess data </h3>

In [ ]:
data_folder = cbs.DATA_PATH / 'hecktor2022'
modalities = [term.Modality.CT, term.Modality.PET]
config_file = cbs.CODEBASE_PATH / 'projects' / 'hecktor2022' / 'experiments' / 'preprocessing.yml'

In [ ]:
hecktor_processor_train = multi_modal_processor.MultiModalProcessor(
    data_folder=data_folder,
    phase=term.Phase.TRAIN,
    modalities=modalities,
    reference=term.Modality.CT,
    config_path=config_file
)

In [ ]:
hecktor_processor_train.preprocess_and_save()

In [ ]:
hecktor_processor_valid = multi_modal_processor.MultiModalProcessor(
    data_folder=data_folder,
    phase=term.Phase.VALID,
    modalities=modalities,
    reference=term.Modality.CT,
    config_path=config_file
)

In [ ]:
hecktor_processor_valid.preprocess_and_save()

In [ ]:
hecktor_processor_test = multi_modal_processor.MultiModalProcessor(
    data_folder=data_folder,
    phase=term.Phase.TEST,
    modalities=modalities,
    reference=term.Modality.CT,
    config_path=config_file
)
hecktor_processor_test.preprocess_and_save()

<h3> Create subvolumes </h3>

In [ ]:
hecktor_processor_train.create_and_save_subvolumes(data_path=data_folder / 'processed_256x256', 
                                             valid_channel=[0], subvolume_intervel=8, subvolume_size=32)

In [ ]:
hecktor_processor_valid.create_and_save_subvolumes(data_path=data_folder / 'processed_256x256', 
                                             valid_channel=[0], subvolume_intervel=8, subvolume_size=32)

In [ ]:
hecktor_processor_test.create_and_save_subvolumes(data_path=data_folder / 'processed_256x256', 
                                             valid_channel=[0], subvolume_intervel=8, subvolume_size=32)

<h3> Debugging </h3>

In [ ]:
label = tio.Image('/workspace/data/hecktor2022/processed_128x128/train/labels/MDA-056.nii.gz')

In [ ]:
idx_slice = 99
print(label.shape)
plt.imshow(label.data[1, :, :, idx_slice])

In [ ]:
import plotly.express as px

In [ ]:
# all_imgs = [label.data[0, :, :, :].numpy(), label.data[1, :, :, :].numpy(), label.data[2, :, :, :].numpy(),]
all_imgs = torch.swapaxes(label.data, 1, 3)
px.imshow(
    all_imgs.numpy(),
    # zmin=[0, 0, 0],
    # zmax=[2000, 2000, 2000],
    animation_frame=1,
    # binary_string=gray_scale,
    labels={'animation_frame': 'slice'},
    facet_col=0,
    color_continuous_scale='Gray',
    width=500*3, height=500
)

In [ ]:
weight = tio.Image('/workspace/data/hecktor2022/processed_128x128/train/labels/MDA-056.nii.gz')

In [ ]:
weight.shape

In [ ]:
filename = 'patient_stats.csv'
columns = ['ID', 'GTVp volume', 'GTVn volume']
hecktor_processor_train.calculate_volumes(data_path=data_folder / 'processed_128x128', output_file=filename, channels=[1, 2], column_names=columns)

In [ ]:
hecktor_processor_valid.calculate_volumes(data_path=data_folder / 'processed_128x128', output_file=filename, channels=[1, 2], column_names=columns)